In [ ]:
# import libararies
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as ras
import os
import glob
from matplotlib  import pyplot as plt
from rasterio.plot import show
import matplotlib
from shapely.geometry import box 
import rioxarray as rxr
from shapely.geometry import mapping
import rasterio.mask as mask

import warnings
warnings.filterwarnings('ignore')


In [ ]:
# required functions
# normalizing the array
def normalize(array):
    """
    normalize an arrary : to prepare to write png
    input: array(any 2D arrray)
    output: noramalized array between 0 -1 
    """
    array_min, array_max = array.min(), array.max()
    return (array - array_min)/(array_max-array_min)

In [ ]:
# image stored location
im_root = '/Users/yahampath/Documents/geospatial/Tools/LNFM/Step3_NAIPOPS/output/before_today'
# list all folders 
im_fold_list = os.listdir(im_root)

In [ ]:
# image stored location
#im_root = '/Users/yahampath/Documents/geospatial/Tools/LNFM/Step3_NAIPOPS/output/before_today'

im_root = '/Users/yahampath/Documents/geospatial/Tools/LNFM/Step3_NAIPOPS/projout/before_today'
# list all folders 
im_fold_list = os.listdir(im_root)
# take the shape file and try to spot the image folder
# list the point files from an editor 
point_list = glob.glob('./input/tylor_points/*shp')
editor = 'Tylor'

# grid list with views
grid_list = []
png_paths = []

for item in point_list:
    # print file path
    print(item)
    # take file name only
    gi_1 = item.split('/')[3]
    
    #gi_1 = item.split('/')[4]
    # create uid only (without year)
    gi_2 = '_'.join(gi_1.split('_')[:4])
    # take year for convential file
    yr1 = gi_1.split('_')[4]
    # it that location has  word point, then conditionals to grab year
    if yr1 == 'point':
        yr1 = gi_1.split('_')[5][0:4]
    else:
        yr1 = yr1
    #print(f'year is {yr1}')
    #  grid id with year 
    grid_id = gi_2 + '_' + yr1
    #print(f'grid id is {grid_id}')
    # append this to a list to built a cat
    grid_list.append(grid_id)
    # grab sample grid number
    grid_id_sub = grid_id.split('_')[3]
    # create canopy file name 
    canopy_file = './input/canopies/'+grid_id+'.shp' # original
    # Create data frame by merging them 
    # have to clip the canopy file to box
    tile = grid_id.split('_')[1]
    sample_grid = gpd.read_file('./input/all_samples_boxes/sampling_grid_'+tile+'.shp')
    # select the box relavant to the trained points
    grid_box  = sample_grid[sample_grid['gridID']==int(grid_id_sub)]
    # open canopy file
    canopy_src = gpd.read_file(canopy_file)
    # open point file
    points_src = gpd.read_file(item)
    # clip the canopy to the box
    canopy_in_grid = gpd.clip(canopy_src,  grid_box)
    # join canopy and point to transfer labeling
    # transfer lables from points to canopy
    canopy_labled = canopy_in_grid.sjoin(points_src, how='left')
    

    #create a color map
    color_map = {
        'Live': '#55EB75', # green
        'Grey': '#EB5570', # red
        'Brown':'#EB5570', # red
        'Partial': '#EBDD55', # yellow
        'NoTree':'#D433F4' # perple
        } 
    
    # listing all image folders
    im_fold_list = os.listdir(im_root)
    im_folder = [item for item in im_fold_list if grid_id_sub in item and tile in item][0]
    print(im_folder)
    # list images in each folder
    image_list = glob.glob(im_root + '/' + im_folder + '/*tif')
    # take image years
    image_names = [item.split('/')[11] for item in image_list]
    #print(image_names)
    image_years = [int(item.split('_')[4][:4]) for item in image_names]
    # drop dupicates
    image_years = list(set(image_years))
    image_years.sort(reverse=True)
    #print(image_years)
    # drop 2004 image and iterate over images to draw pallates; some how call the specific year data 
    # column/row as well using the status or year
    # then game over

    # read NAIP years to a list
    naip_years = []
    for col in ['NAIP1', 'NAIP2', 'NAIP3', 'NAIP4', 'NAIP5', 'NAIP6', 'NAIP7', 'NAIP8', 'NAIP9', 'NAIP10']:
        year_np = canopy_labled[col].dropna().unique()
        if len(year_np) == 1:
            naip_years.append(year_np[0])

    naip_years = [int(item) for item in naip_years if 'NA' not in item]
    #print(naip_years)
    num_of_images = len(naip_years)
    
    # create a buffer to box to clip the naip image for display
    # create new box buffer for the gridbox
    #buffer_distance = 35
    #grid_box['buffer'] = grid_box['geometry'].apply(lambda square: box(square.centroid.x - buffer_distance, #square.centroid.y - buffer_distance, square.centroid.x + buffer_distance, square.centroid.y + #buffer_distance))
    #new_box = gpd.GeoDataFrame(grid_box, geometry='buffer')


    print(num_of_images)

    # empty list for images
    draw_paths = []

    counter = 0
    # Set to keep track of years for which an image path has already been selected
    selected_years = set()
    # Iterate through the years
    for year in naip_years:
        # Check if we have already selected an image for this year
        if year in selected_years:
            continue

        #print(year)
        # Counter for plot number
        counter += 1

        for image_path in image_list:
            if str(year) in image_path:
                #print(image_path)
                # add image path to the drawing list
                draw_paths.append(image_path)
                # Add the year to the set to avoid selecting more images for this year
                selected_years.add(year)
                break  # Stop after finding the first image for this year

                # print  list of drawing path
    #print(draw_paths)
    #print(naip_years)

    # above this line there is no need of edits

    # potential column set to slice larger data frame for the yearly
    status_cols = ['status1', 'status2', 'status3', 'status4', 'status5', 'status6', 'status7', 'status8', 'status9', 'status10']
    naip_cols = ['NAIP1', 'NAIP2', 'NAIP3', 'NAIP4', 'NAIP5', 'NAIP6', 'NAIP7', 'NAIP8', 'NAIP9', 'NAIP10']

    # decide the number of rows and columns for the image based on drawpath list lenght ???

    # take the first image of imagelist to extract crs: will use to reproject other layers
    crs_im_path = image_list[0]
    crs_image = ras.open(crs_im_path)
    use_crs = crs_image.crs

    # main figure
    plt.figure(figsize=(12,15))


    counter_1 = 0
    for i, j in zip(draw_paths, naip_years):
        #print(i)
        #print(j)
        df_year = canopy_labled[[naip_cols[counter_1],
                                 status_cols[counter_1],
                                  'geometry' ]]
        df_year['color'] = df_year[status_cols[counter_1]].map(lambda x: color_map.get(x, 'white'))  
        counter_1 += 1
        
        #print(df_year.crs)

        # now open the image for each year
        buffer_distance = 35
        grid_box['buffer'] = grid_box['geometry'].apply(lambda square: box(square.centroid.x - buffer_distance, square.centroid.y - buffer_distance, square.centroid.x + buffer_distance, square.centroid.y + buffer_distance))

        new_box = gpd.GeoDataFrame(grid_box, geometry='buffer')
        new_box = new_box.to_crs(use_crs)
        grid_box  = grid_box.to_crs(use_crs)


        # image open
        image_src = rxr.open_rasterio(i)
        # clip image with new box
        rgb_clip = image_src.rio.clip(new_box.geometry.apply(mapping), new_box.crs)
        # save the clip before work with raster 
        rgb_clip.rio.to_raster('rgb_'+str(j)+'.tif', driver='GTiff')

        plt.subplot(4,3,counter_1)
        # read image and bands
        src = ras.open('rgb_'+str(j)+'.tif')
        red  = src.read(1)
        green  = src.read(2)
        blue = src.read(3)
        # normalize
        redn = normalize(red)
        greenn = normalize(green)
        bluen = normalize(blue)
        # stack bands
        rgb = np.dstack((redn, greenn, bluen))
        extent = src.bounds
        image_crs = src.crs

        df_year = df_year.to_crs(use_crs)
        grid_box = grid_box.to_crs(use_crs)

        rgb_disp = plt.imshow(rgb, extent=[extent.left, extent.right,extent.bottom, extent.top]
                                            #,interpolation='bilinear'
                                           )
        
       
        df_year.boundary.plot(ax=plt.gca(), color=df_year['color'], linewidth=1)
        grid_box.boundary.plot(ax=plt.gca(), color='#2874A6', linewidth=1)

        # year
        plt.title('Year-'+str(j), fontsize=10, color='#2a3240', fontweight='bold') 
        plt.gca().set_facecolor('none')
        plt.axis('off')

    plt.suptitle(grid_id + ' ' + 'Edited by - '+editor, fontsize=18, color='#28A695', fontweight='bold')
    plt.tight_layout()
    #plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.2, wspace=0.2)
    plt.savefig('./output/images/'+grid_id+'.png')
    plt.show()
    plt.close()
    


In [ ]:
ls = glob.glob('./output/images/*png')
len(ls)

In [ ]:
# create an empty data frame
qc_views_df = pd.DataFrame()

qc_views_df['grid_id'] = grid_list
qc_views_df['Editor'] = 'Tylor'
qc_views_df['day'] = '20241015'
qc_views_df

In [ ]:
# save data frame to a csv
qc_views_df.to_csv('./output/cats/catalogue_20241016.csv')